Deutsch-Josza algorithm for n qubits - qasm

In [4]:
import qiskit 
from qiskit import *
from qiskit_aer import AerSimulator 
import matplotlib as plt
from random import choice, randint 

import json
import requests 


In [16]:
n = 15

qasm_circuit = []
qasm_circuit.append("OPENQASM 2.0;")
qasm_circuit.append('include "qelib1.inc";')
qasm_circuit.append(f"qreg q[{n+1}];")  # q[0..n-1] query, q[n] ancilla
qasm_circuit.append(f"creg c[{n}];")    # measure query only
qasm_circuit.append("")
# Prepare superposition on query, and |-> on ancilla: X then H on ancilla

# (A) Prepare |+...+> on query
for i in range(n):
    qasm_circuit.append(f"h q[{i}];")

qasm_circuit.append(f"x q[{n}];")
qasm_circuit.append(f"h q[{n}];")
qasm_circuit.append("barrier q;")
qasm_circuit.append("")
  

In [17]:
def Oracle(n, djc):
    """
    Returns (qasm_string, oracle_type, a_bits_used).
    Oracle types:
      - Constant0: do nothing
      - Constant1: X on ancilla
      - Balanced : CNOTs from query qubits where a_bits[i]=='1' to ancilla
                 (implements f(x)=a·x mod 2, balanced for any nonzero a)
    """
    oracle_type = choice(['Constant0', 'Constant1', 'Balanced'])

    # Oracle
    if oracle_type == "Constant0":
        pass
    elif oracle_type == "Constant1":
        djc.append(f"x q[{n}];")
    elif oracle_type == "Balanced":
        a = randint(1, 2**n - 1)
        a_bits = format(a, f"0{n}b")
        for i, bit in enumerate(a_bits):
            if bit == "1":
                djc.append(f"cx q[{i}],q[{n}];")
    
    return djc, oracle_type
  


In [ ]:
lines, oracle_type = Oracle(n, qasm_circuit)

lines.append("barrier q;")
lines.append("")

# Interference + measurement (query only)
for i in range(n):
    lines.append(f"h q[{i}];") 
for i in range(n):
    lines.append(f"measure q[{i}] -> c[{i}];")

#create the final text you send to quokka 
qasm = "\n".join(lines) + "\n"

#drawing the circuit
qc = QuantumCircuit.from_qasm_str(qasm)
print("\n=== Circuit diagram ===")
print(qc.draw("text"))   # use "mpl" if you want matplotlib drawing

# Example usage (n=30):
print("Answer Oracle:", oracle_type)
print(qasm[:500], "...\n")  # the 500 means the first 500 characters of the qasm text




=== Circuit diagram ===
      ┌───┐      ░  ░ ┌───┐┌─┐                                          
 q_0: ┤ H ├──────░──░─┤ H ├┤M├──────────────────────────────────────────
      ├───┤      ░  ░ ├───┤└╥┘┌─┐                                       
 q_1: ┤ H ├──────░──░─┤ H ├─╫─┤M├───────────────────────────────────────
      ├───┤      ░  ░ ├───┤ ║ └╥┘┌─┐                                    
 q_2: ┤ H ├──────░──░─┤ H ├─╫──╫─┤M├────────────────────────────────────
      ├───┤      ░  ░ ├───┤ ║  ║ └╥┘┌─┐                                 
 q_3: ┤ H ├──────░──░─┤ H ├─╫──╫──╫─┤M├─────────────────────────────────
      ├───┤      ░  ░ ├───┤ ║  ║  ║ └╥┘┌─┐                              
 q_4: ┤ H ├──────░──░─┤ H ├─╫──╫──╫──╫─┤M├──────────────────────────────
      ├───┤      ░  ░ ├───┤ ║  ║  ║  ║ └╥┘┌─┐                           
 q_5: ┤ H ├──────░──░─┤ H ├─╫──╫──╫──╫──╫─┤M├───────────────────────────
      ├───┤      ░  ░ ├───┤ ║  ║  ║  ║  ║ └╥┘┌─┐                        
 q_6: ┤ H ├──────░──░─┤ H 

In [19]:
import urllib3
urllib3.disable_warnings()

#my_quokka = "theq-adc1cf" 
my_quokka = 'quokka1'


In [ ]:
request_http = f"http://{my_quokka}.quokkacomputing.com/qsim/qasm"

data = {
    "script": qasm,
    "count": 10
}

result = requests.post(request_http, json=data, verify=False)


#print("HTTP status:", result.status_code)
#print("Content-Type:", result.headers.get("Content-Type"))
#print("First 300 chars of body:\n", result.text[:300])

counts = json.loads(result.content)

print("Raw counts:",counts)
   

Raw counts: {'error': 'no error', 'error_code': 0, 'result': {'c': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}}


In [21]:
shots_bits = counts["result"]["c"]   # appears to be [shots][bits]
if all(all(b == 0 for b in shot) for shot in shots_bits):
    print("Prediction: Constant")
else:
    print("Prediction: Balanced")

print("Truth: ", oracle_type)


Prediction: Constant
Truth:  Constant0
